Start Chrome from the command line with a CDP port. Open a terminal and run:

    "C:\Program Files\Google\Chrome\Application\chrome" --remote-debugging-port=9222 --user-data-dir="D:\Desktop\Nivea-Assignment\shopee-scrape\browserData"

Check for the WebSocket

    http://127.0.0.1:9222/json/version

In [1]:
from playwright.async_api import async_playwright, CDPSession
from playwright_stealth import stealth_async
import requests
import time

p = await async_playwright().start()

# # app 1
def existing_browser():
    ws_url = "ws://127.0.0.1:9222/devtools/browser/2f2d448f-382d-4cce-9a33-32798f70dc83"
    return ws_url

# # app 3
def subprocess_browser():
    import subprocess

    chrome_exe_path = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"
    browser_data_dir = "D:\\Desktop\\Nivea-Assignment\\shopee-scrape\\browserData"
    
    process = subprocess.Popen([
        chrome_exe_path,
        "--remote-debugging-port=9222",
        f"--user-data-dir={browser_data_dir}",
    ])
    time.sleep(5)
    class KillProcess:
        def run():
            process.terminate()
            process.wait()
    ws_data = requests.get("http://localhost:9222/json/version").json()
    ws_url = ws_data["webSocketDebuggerUrl"]
        
    return ws_url, KillProcess
    
# # app 2
async def playwright_browser():
    chrome_exe_path = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"
    browser_data_dir = "D:\\Desktop\\Nivea-Assignment\\shopee-scrape\\browserData"
    await p.chromium.launch_persistent_context(
                executable_path=chrome_exe_path,
                user_data_dir=browser_data_dir,
                headless=False,
                args=[
                    "--remote-debugging-port=9222",
                ]
            )
    ws_data = requests.get("http://localhost:9222/json/version").json()
    ws_url = ws_data["webSocketDebuggerUrl"]
    return ws_url
    
ws_url, terminator = subprocess_browser()

browser = await p.chromium.connect_over_cdp(ws_url)
context = browser.contexts[0]
page = context.pages[0]
# await stealth_async(page) #! Actually make the browser more detectable

globs = {
    "session": [],
    "cmds": [],
    "dataIds": [],
    }

def gwrap_session(session : CDPSession):
    globs['session'].append(session)
    
def gwrap_(func):
    globs['cmds'].append(func)

print("Connected to browser over CDP.")


# Enable the Network domaine
gwrap_session( session := await context.new_cdp_session(page) )
gwrap_( await session.send("Network.enable") )


# Change monitor source

import sys
errors = open("errors.log", "a")
sys.stderr = errors

print("This will go nowhere")

Connected to browser over CDP.
This will go nowhere


### UI Interactive flow

- Navigate to Target site
- Perform human validation (if needed)
- Set a listener on Ajax repsonse
- Interact with the site to go invoke Ajax retrieval
- Repeat up to a limit

In [7]:
import json
TERM = 'search_item'

def clear_listeners(event):
    del session._impl_obj._events[event]
    with open("URL.log", "a") as f:
        f.write(f"Listeners cleared for {event}\n")
    return session._impl_obj._events.get(event, None)

def calculate_page_number(url):
    import re
    newest_match = re.search(r'newest=(\d+)', url)
    limit_match = re.search(r'limit=(\d+)', url)
    if not newest_match or not limit_match:
        return None
    newest = int(newest_match.group(1))
    limit = int(limit_match.group(1))
    page_number = newest / limit + 1
    return page_number

async def collect_data(repsonse):
    with open("URL.log", "a") as ff:
        ff.write(f"\nCollecting data on: {repsonse['requestId']}\n\n")
        pg_num = calculate_page_number(repsonse['response']['url'])
        time.sleep(2)
        with open(f"data/page_{int(pg_num)}.json", "w", encoding='utf-8') as f:
            for i in range(5):
                ff.write(f"Attempt Collecting: {i+1} at requestId: {repsonse['requestId']}")    
                # print(f"Attempt Collecting: {i+1} at requestId: {repsonse['requestId']}")    
                result = await session.send("Network.getResponseBody", { "requestId": repsonse['requestId'] })
                if result: break
            # f.write(f"\nRequestID: {repsonse['requestId']}\nResponse: {result['body']}\n\n") #? Content data tested and writable
            body = json.loads(result['body'])
            f.write(json.dumps(body, indent=4))
    return True

async def upon_response_received(response):
    url = response["response"]["url"]
    requestId = response['requestId']
    
    with open("URL.log", "a") as f:
        f.write(f"Network.responseReceived: {requestId}\nURL: {url}\n")
        if TERM in url:    
            f.write(f'\nSearch_Item Response\nRequestID: {requestId}\nURL: {url}\n\n')
            globs['dataIds'].append(requestId)
            await collect_data(response)
            return True
    return False

# gwrap_( session.on("Network.requestWillBeSent", upon_response) )
# gwrap_( session.on("Network.loadingFinished", collect_data_then_mute) )
gwrap_( session.on("Network.responseReceived", upon_response_received) )

In [8]:
# Check for session listeners and Get rid of listeners explicitly
# clear_listeners("Network.loadingFinished")
# clear_listeners("Network.requestWillBeSent")

# clear_listeners("Network.responseReceived")
[ print ( i, f'listeners={len(j)}', [item for item in j] ) for i,j in session._impl_obj._events.items() ]

Network.responseReceived listeners=1 [<function ImplToApiMapping.wrap_handler.<locals>.wrapper_func at 0x000001C540ACFC40>]


[None]

In [ ]:
# test getResponseBody
requestId = "18280.1308"
result = await session.send("Network.getResponseBody", { "requestId": requestId })
with open("response.log", "a", encoding='utf-8') as f:
    f.write(f"\nRequestID: {requestId}\nResponse: {result['body']}\n") #? Content data tested and writable

In [9]:
# Click the buttons
for i in range(10):
    # Use CDP to execute the script within the page context
    result = await session.send("Runtime.evaluate", {
        "expression": """document.getElementsByClassName("shopee-button-outline shopee-mini-page-controller__next-btn")[0].click()""",
        "returnByValue": True
    })
    time.sleep(5.264)
    with open("URL.log", "a") as f:
        f.write(f"Clicking Next Button: {i+1}\n")

In [ ]:
async def data_retrieval():
    for id, i in zip(globs['dataIds'], range(len(globs['dataIds']))):
        result = await session.send("Network.getResponseBody", {
            "requestId": id
        })
        with open(f"data/page_{i}.json", "w") as f:
            f.write(result["body"])

In [10]:
# Close Browser and Playwright
if session: 
    await session.detach()
    print ("Session detached.")
await browser.close()
await p.stop()
terminator.run()
print("Browser closed and Playwright stopped.")

# Revert monitor settings
sys.stderr = sys.__stderr__

# Close the file
errors.close()

Session detached.
Browser closed and Playwright stopped.


# Utility

In [ ]:
# Detach session to get rid of listeners and create new session
async def recreate_session():
    gwrap_( session.send("Network.disable") )
    gwrap_( session.detach() )
    gwrap_session(session := await context.new_cdp_session(context.pages[0]) )
    gwrap_( await session.send("Network.enable") )

# recreate_session()
#* not working at all
# gwrap_( session.remove_listener("Network.responseReceived", upon_response) )

In [ ]:
# TESTING STEALTH MODE
page = await browser.new_page()
stealth_async(page)  # Apply stealth mode
print("Stealth techniques applied to the first page.")

page.goto("https://arh.antoinevastel.com/bots/areyouheadless")
message_element = page.locator("#res")
message = message_element.text_content()
print(f'The result is: "{message}"')